In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.cluster import DBSCAN

# Expected centers for the images
expected_centers = {
    "10c.tif": (1305, 1014),
    "11b.tif": (1224, 996),
    "14a.tif": (1257, 1089),
    "20c.tif": (1143, 921),
    "36a.tif": (1491, 1125),
}

# Hyperparameter ranges
param_ranges = {
    "dp": [1.0, 1.25, 1.5, 1.75, 2.0],
    "minDist": [30, 50, 70],
    "param1": [50, 100, 150],
    "param2": [20, 30, 40],
    "eps": [50, 100, 150],
    "min_samples": [3, 5, 7],
}

# Number of random combinations to try
n_iterations = 50

# Function to calculate error
def calculate_error(detected_center, expected_center):
    if detected_center is None:
        return float("inf")
    return np.sqrt((detected_center[0] - expected_center[0]) ** 2 + (detected_center[1] - expected_center[1]) ** 2)

# Random search
best_params = {}
accuracy_metrics = {}

for image_name, expected_center in expected_centers.items():
    # Load image
    image_path = f"data/PSF0001_{image_name}"  # Update path as necessary
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Failed to load image: {image_name}")
        continue

    # Initialize best error
    best_error = float("inf")
    best_hyperparams = None

    # Perform random search
    for _ in range(n_iterations):
        # Randomly sample hyperparameters
        dp = random.choice(param_ranges["dp"])
        minDist = random.choice(param_ranges["minDist"])
        param1 = random.choice(param_ranges["param1"])
        param2 = random.choice(param_ranges["param2"])
        eps = random.choice(param_ranges["eps"])
        min_samples = random.choice(param_ranges["min_samples"])

        # Edge detection
        edges = cv2.Canny(image, threshold1=param1, threshold2=param2)

        # Hough Circle Transform
        circles = cv2.HoughCircles(
            edges,
            cv2.HOUGH_GRADIENT,
            dp=dp,
            minDist=minDist,
            param1=param1,
            param2=param2,
            minRadius=5,
            maxRadius=50,
        )

        if circles is not None:
            circles = np.uint16(np.around(circles))
            detected_centers = [(circle[0], circle[1]) for circle in circles[0, :]]
            points = np.array(detected_centers)

            # DBSCAN clustering
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(points)

            # Find cluster centroids
            unique_labels = set(labels) - {-1}
            cluster_centers = [
                np.mean(points[labels == label], axis=0)
                for label in unique_labels
            ]

            # Calculate final center (midpoint of all cluster centroids)
            if len(cluster_centers) > 0:
                center_x = int(np.mean([c[0] for c in cluster_centers]))
                center_y = int(np.mean([c[1] for c in cluster_centers]))
                final_center = (center_x, center_y)
            else:
                final_center = None

            # Calculate error
            error = calculate_error(final_center, expected_center)

            # Update best parameters if error improves
            if error < best_error:
                best_error = error
                best_hyperparams = {
                    "dp": dp,
                    "minDist": minDist,
                    "param1": param1,
                    "param2": param2,
                    "eps": eps,
                    "min_samples": min_samples,
                }

    # Save best parameters and accuracy
    best_params[image_name] = best_hyperparams
    accuracy_metrics[image_name] = {
        "error": best_error,
        "expected_center": expected_center,
        "detected_center": final_center,
    }

# Display results
print("Optimal Hyperparameters for Each Image:")
for image_name, params in best_params.items():
    print(f"{image_name}: {params}")

print("\nAccuracy Metrics:")
for image_name, metrics in accuracy_metrics.items():
    print(f"{image_name}: {metrics}")


Optimal Hyperparameters for Each Image:
10c.tif: {'dp': 1.0, 'minDist': 30, 'param1': 50, 'param2': 40, 'eps': 50, 'min_samples': 5}
11b.tif: {'dp': 1.25, 'minDist': 30, 'param1': 100, 'param2': 40, 'eps': 100, 'min_samples': 7}
14a.tif: {'dp': 1.5, 'minDist': 70, 'param1': 50, 'param2': 20, 'eps': 150, 'min_samples': 7}
20c.tif: {'dp': 1.5, 'minDist': 70, 'param1': 100, 'param2': 20, 'eps': 100, 'min_samples': 7}
36a.tif: {'dp': 1.75, 'minDist': 50, 'param1': 150, 'param2': 40, 'eps': 150, 'min_samples': 7}

Accuracy Metrics:
10c.tif: {'error': 24.020824298928627, 'expected_center': (1305, 1014), 'detected_center': None}
11b.tif: {'error': 36.124783736376884, 'expected_center': (1224, 996), 'detected_center': (1285, 977)}
14a.tif: {'error': 94.30270409696638, 'expected_center': (1257, 1089), 'detected_center': None}
20c.tif: {'error': 40.607881008493905, 'expected_center': (1143, 921), 'detected_center': (1673, 1476)}
36a.tif: {'error': 45.27692569068709, 'expected_center': (1491, 112